## Exploration Notebook

### Goal is to identify issues in the data set

1. General Overview
2. Inspect each column and identify possible solutions
3. Summary of Identified Issues
4. Implementation of reusable methods in seperate skripts

In [3]:
# Import libraries and data
import pandas as pd
raw_dataset = pd.read_csv("data/interview_signup.csv", sep=',')


## 1. General Overview

In [4]:
raw_dataset.head()

,original_product_name,postcode,bundesland,total_bonus,order_date
0,E.ON STROM,53229.0,Nordrhein-Westfalen,146.0,2018-01-01
1,E.ON STROM ÖKO,74235.0,Baden-Württemberg,67.0,2018-01-01
2,E.ON STROM ÖKO 24,45257.0,Nordrhein-Westfalen,123.0,2018-01-01
3,E.ON STROM,64395.0,Hessen,159.0,2018-01-01
4,E.ON STROM 24,36039.0,Hessen,116.0,2018-01-01


In [5]:
raw_dataset.tail()

,original_product_name,postcode,bundesland,total_bonus,order_date
318340,E.ON STROM 24,49610,Niedersachsen,147.0,2018-12-31
318341,E.ON STROM,4916.0,NaN,142.0,2018-12-31
318342,E.ON STROM,77972.0,Baden-Württemberg,48.0,2018-12-31
318343,E.ON STROM ÖKO 24,91466,Bayern,238.0,2018-12-31
318344,E.ON STROM PUR,88145,Bayern,125.0,2018-12-31


### Explanation: Python already hints that there are mixes data types in the postcode columns. Decimal numbers already look suspicipus for postcodes. Also Null values appear in bundesland

In [6]:
#get basic info about data

raw_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 318345 entries, 0 to 318344
Data columns (total 5 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   original_product_name  318345 non-null  object 
 1   postcode               318345 non-null  object 
 2   bundesland             288813 non-null  object 
 3   total_bonus            318345 non-null  float64
 4   order_date             318345 non-null  object 
dtypes: float64(1), object(4)
memory usage: 12.1+ MB


### Explanation: Confirms Null values in 'bundesland' column. Rest do not seem to have missing values

In [7]:
# basic statistics to get a better overview
raw_dataset.describe()

,total_bonus
count,318345.000000
mean,150.590275
std,53.692176
min,0.000000
25%,112.000000
50%,149.000000
75%,182.000000
max,570.000000


In [8]:
raw_dataset.describe(include=object)

,original_product_name,postcode,bundesland,order_date
count,318345,318345.0,288813,318345
unique,10,20525.0,16,365
top,E.ON STROM,31515.0,Nordrhein-Westfalen,2018-09-05
freq,164473,332.0,68541,960


### Explanation: Order Date seems to be in order (365 unique values). Bundesland also seems ok. Reason for Null values still unclear

## 2. Inspect each column and identify possible solutions

## original product name

In [9]:
raw_dataset['original_product_name'].value_counts()

E.ON STROM                         164473
E.ON STROM 24                       69363
E.ON STROM ÖKO                      55194
E.ON STROM ÖKO 24                   25440
E.ON STROM PUR                       3650
E.ON STROM Ã–KO                       206
E.ON STROM 24 24                       14
E.ON STROM 24 24 24                     2
E.ON STROM 24 24 24 24 24 24 24         2
E.ON STROM ÖO                           1
Name: original_product_name, dtype: int64

In [10]:
225/318345

0.0007067803797766574

### Explanation: Some names do not make sense. Needs to be dicussed with business and identified in Data Source system where the name come from. Since its only a very small percantage of data, <1%, these datapoints could be eliminated until the names are corrected

## order_date

In [11]:
raw_dataset[(raw_dataset['order_date'] > '2018-12-31') | (raw_dataset['order_date'] < '2018-01-01')]

,original_product_name,postcode,bundesland,total_bonus,order_date


### Explanation: Seems to be in order. Nothing to clean

## postcode and bundesland

In [12]:
raw_dataset[['postcode']]

,postcode
0,53229.0
1,74235.0
2,45257.0
3,64395.0
4,36039.0
...,...
318340,49610
318341,4916.0
318342,77972.0
318343,91466


### Explanation: Value 318341 seems interesting. It is not quite clear if 4916.0 is the same as 49160 or if it is its own postcode 04916

In [13]:
# Further investigation of 4 and 5 digit postcodes. First get rid of the decimal 

postcodes_as_strings = raw_dataset['postcode'].astype('str')
postcodes_without_decimal = [elem.split('.')[0]  if '.' in elem else elem for elem in postcodes_as_strings]
raw_dataset['postcodes_without_decimal'] = postcodes_without_decimal
raw_dataset.head()


,original_product_name,postcode,bundesland,total_bonus,order_date,postcodes_without_decimal
0,E.ON STROM,53229.0,Nordrhein-Westfalen,146.0,2018-01-01,53229
1,E.ON STROM ÖKO,74235.0,Baden-Württemberg,67.0,2018-01-01,74235
2,E.ON STROM ÖKO 24,45257.0,Nordrhein-Westfalen,123.0,2018-01-01,45257
3,E.ON STROM,64395.0,Hessen,159.0,2018-01-01,64395
4,E.ON STROM 24,36039.0,Hessen,116.0,2018-01-01,36039


### I am curious if there are postcodes that have a missing 'bundesland' and defined bundesland for postcode length 4

In [14]:
filtered_postcodes_with_four_digits = raw_dataset[(raw_dataset['bundesland'].isna() | raw_dataset['bundesland'].notna()) & (raw_dataset['postcodes_without_decimal'].str.len() == 4)]
unique_postcode_bundeland_pairs = set(zip(filtered_postcodes_with_four_digits['postcodes_without_decimal'],filtered_postcodes_with_four_digits['bundesland']))
#unique_postcode_bundeland_pairs


### Explanation: There are indeed 4 digit postcodes that habe a bundesland and also NAs. These can be matched to fill NAs. This can then also be done with 5 digits

### The question remains if four digit postalcodes should all get a leading zero as they are their own 'zone' or if the decimal point seperated the postalcode 

In [15]:
listOfShortCodes = list(raw_dataset[(raw_dataset['bundesland'].isna()) & (raw_dataset['postcodes_without_decimal'].str.len() == 4)]['postcodes_without_decimal'])
listOfCorrectCodes = list(set(raw_dataset[(raw_dataset['bundesland'].isna()) & (raw_dataset['postcodes_without_decimal'].str.len() == 5)]['postcodes_without_decimal']))

In [ ]:
# test with a regex expression if 4 digits codes appear in 5 digits codes with leading or tailing zeros. This could determine how postcodes can be cleaned
import re

for element in listOfShortCodes:
    r = re.compile(".*"+element+".*")
    print(element)
    list_where_matched = list(filter(r.match, listOfCorrectCodes))
    print(list_where_matched)

### Explanation: The comparision of 4 and 5 digit postal codes shows that we can not conclude that 4916.0 is equal to 49160. In contrast, a quick research of postcodes showed that the four digit codes with a leading 0 makes more sense (these are usualy zones in eastern germany). To be sure this needs to be confirmed in the data source, otherwise it will lead to wrong data points.

# 3. Summary of issues

### I have identified several issues in the data. I decide to implement 2 methods. One will match NaN values in bundesland based on postcodes (if the postcode/bundesland combination already appeared). The second method will add a leading zero to postcodes so all codes have 5 digits.

# 4. Implementation

### see src/cleaning_helpers.py

### Validation

In [23]:
# Import helper functions

from src.cleaning_helpers import clean_postcodes as cp
from src.cleaning_helpers import match_postcodes_to_bundesland as mtb
import pandas as pd


In [24]:
# Constants

DATA_PATH = 'data/interview_signup.csv'

In [26]:
# Import data
df = pd.read_csv(DATA_PATH, sep=',')


/Users/admininistrator/eon.datachallenge/env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [27]:
# Clean Postcodes

df_cleaned_postcodes = cp(df)
df_cleaned_postcodes

,original_product_name,postcode,bundesland,total_bonus,order_date
0,E.ON STROM,53229,Nordrhein-Westfalen,146.0,2018-01-01
1,E.ON STROM ÖKO,74235,Baden-Württemberg,67.0,2018-01-01
2,E.ON STROM ÖKO 24,45257,Nordrhein-Westfalen,123.0,2018-01-01
3,E.ON STROM,64395,Hessen,159.0,2018-01-01
4,E.ON STROM 24,36039,Hessen,116.0,2018-01-01
...,...,...,...,...,...
318340,E.ON STROM 24,49610,Niedersachsen,147.0,2018-12-31
318341,E.ON STROM,04916,NaN,142.0,2018-12-31
318342,E.ON STROM,77972,Baden-Württemberg,48.0,2018-12-31
318343,E.ON STROM ÖKO 24,91466,Bayern,238.0,2018-12-31


In [ ]:
# match missing bundeslaender

df_matched_bundesland = mtb(df_cleaned_postcodes)
df_matched_bundesland

In [ ]:
df_matched_bundesland.isna().sum()

In [ ]:
raw_dataset.isna().sum()

### About 7000 entries could be matched